# Запуск сервера

In [ ]:
#@title Установка и импорт библиотек
!pip  install  tiktoken langchain openai faiss-cpu gspread oauth2client nltk pydantic openpyxl fastapi requests pytz python-dotenv ngrok pyngrok uvicorn kaleido python-multipart cohere

In [ ]:
# добавить в Секрет Колаба (ключ слева на панели) переменные окружения Authtoken, SECRET_OPENAI_KEY, git_token
from google.colab import userdata
git_token = userdata.get('git_token')

In [ ]:
# или через getpass
# from getpass import getpass
# git_token = getpass()
# SECRET_OPENAI_KEY = getpass()
# Authtoken = getpass()

In [ ]:
import os
os.environ['SECRET_OPENAI_KEY'] = userdata.get('SECRET_OPENAI_KEY')
# os.environ['SECRET_OPENAI_KEY'] = SECRET_OPENAI_KEY

In [ ]:
Authtoken = userdata.get('Authtoken')
# !ngrok config add-authtoken {Authtoken}
!ngrok authtoken {Authtoken}

In [ ]:
!git clone -b develop https://{git_token}@github.com/your_project.git

In [ ]:
# !pip install -r requerements

In [ ]:
%cd /content/your_project/

In [ ]:
!cp example.env .env

In [ ]:
!nohup uvicorn main:app --reload &

In [ ]:
# !nohup uvicorn main:app --reload --host "0.0.0.0" --port "8000" &   # можно еще такой вариант потестить

In [ ]:
from pyngrok import ngrok

ngrok.kill()
connection = ngrok.connect(8000)      # запускаем туннель на побличный адрес из локолхоста порт 8000
print(f"Public URL: {connection.public_url}")

# Если сервер с GPT уже запущен, то начинаем отсюда!!!

In [ ]:
# проверяем работу
url = connection.public_url   # укажите адрес сервера

In [ ]:
import requests
response = requests.get(url)    # проверка GET запроса

In [ ]:
print(response.status_code)
print(response.text)

### Проверяем работу с запросом

In [ ]:
message = "привет"

In [ ]:
# Создайте словарь с данными, которые вы хотите отправить
data = {
    "message": message,
    "temperatura": "0",
    "doc_count": "3",
    "new_dialog": "0"
}

In [ ]:
# data = {
#     "message": "hi"
# }

In [ ]:
# Преобразуйте словарь в JSON
import json
json_data = json.dumps(data)

In [ ]:
json_data

In [ ]:
url = f"{connection.public_url}/message"   # укажите адрес сервера

In [ ]:
# Отправьте POST-запрос
response = requests.post(url, data=json_data, headers={"Content-Type": "application/json"})

In [ ]:
# проверка, является ли ответ json
# if 'application/json' in response.headers['Content-Type']:
#     response_json = response.json()
# else:
#     print('Response is not JSON')

In [ ]:
response

In [ ]:
# Выведите ответ
print(response.status_code)
print(response.json())

In [ ]:
response_json = response.json()
answer_text = response_json['answer']['text']
print(answer_text)

In [ ]:
import re
import requests
from io import BytesIO
import openpyxl
import pandas as pd
from tqdm.auto import tqdm
import IPython

In [ ]:
# @title функция для загрузки таблицы по ссылке из гугл драйв
#def load_xls_pd(url: str) -> str:
def load_xls_pd(url: str, sheet_name: str = 0) -> str:

    # Extract the document ID from the URL
    match_ = re.search('/spreadsheets/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Sheets URL')
    doc_id = match_.group(1)

    if sheet_name is None:
        raise ValueError('Invalid sheet_name')

    # Download the table as pandas
    response = requests.get(f'https://docs.google.com/spreadsheets/d/{doc_id}/export?format=xlsx')
    response.raise_for_status()     #проверяет статус код ответа. Если получен ответ с кодом ошибки (4xx или 5xx), вызывается исключение HTTPError.
    #data = pd.read_excel(BytesIO(response.content), sheet_name=0)  #чтение первого листа из книги
    data = pd.read_excel(BytesIO(response.content),sheet_name = sheet_name, header=None) #чтение конкретного листа из Книги Excel

    return data

In [ ]:
# @title Загрузка таблицы для тестирования
# Шаг 1: Загрузить xlsx файл из Google Drive
google_sheet_url = "https://docs.google.com/spreadsheets/d/1U4J6GgcamnRmV7mjhcbXxhTIsJwOVyKPqAVnimcHtR0/edit?usp=sharing"
sheet_name='Вопросы для тестирования'     # вставить название листа для заполнения вопросами

#data = load_xls_pd(google_sheet_url)   #если в таблице только один лист
df = load_xls_pd(google_sheet_url, sheet_name)

In [ ]:
# проверка
df.head()

In [ ]:
# установить первую строку в виде строки заголовков столбцов
df.set_axis(df.iloc[0], axis='columns', inplace=True)
# проверка
df.info()
df.head()


### Сохранение ответов ChatGPT и чанков в таблицу



In [ ]:
# Переменная для подсчета количества строк, обработанных по условию
count = 0
print('Количество обработанных запросов: ')
# Цикл для обхода строк DataFrame
for index, row in tqdm(df.iterrows()):
    # if (row['Тема'] == 'Аксесуары') & (pd.isna(row['Ответ GPT'])):       # Выбираем строки со своей Темой для тестирования, и если в строке еще нет ответа
    if (pd.isna(row['Ответ GPT'])):       # Выбираем строки если в строке еще нет ответа
        # Ваш код для обработки данных текущей строки
        # Создайте словарь с данными, которые вы хотите отправить
        message = row['Вопрос']
        data = {
            "message": message,
            "temperatura": "0",
            "doc_count": "3",
            "new_dialog": "0"
        }
        # Преобразуйте словарь в JSON
        json_data = json.dumps(data)

        # Отправьте POST-запрос
        response = requests.post(url, data=json_data, headers={"Content-Type": "application/json"})

        # записываем ответ в соответствующий столбец
        response_json = response.json()
        answer = response_json['answer']['text']
        urls = response_json['answer']['urls']
        answer_time = response_json['answer']['answer_time']
        # chanks = response_json['answer']['chanks']
        # Записываем результат в столбец "Ответ"
        df.at[index, 'Ответ GPT'] = answer
        # Записываем чанки
        # df.at[index, 'Чанки'] = chanks
        # Записываем чанки
        df.at[index, 'Ссылки'] = urls
        # Записываем чанки
        df.at[index, 'Время ответа'] = answer_time

        # Увеличиваем значение счетчика и проверяем, достигли ли мы нужного количества запросов
        count += 1
        print(count, end=' ')
        if count >= 5:          # количество задаваемых вопросов к ChatGPT
            break


Количество обработанных запросов: 


0it [00:00, ?it/s]

1 2 3 4 5 

In [ ]:
# df[df['Тема'] == 'Аксесуары'][['Вопрос', 'Ответ GPT', 'Чанки']].head(15)   #проверка записей в таблице
df[['Вопрос', 'Ответ GPT', 'Ссылки', 'Время ответа', 'Чанки']].head(10)   #проверка записей в таблице

In [ ]:
# df[(df['Тема'] == 'Аксесуары') & (df['Ответ GPT'].isna())].head(10) #проверка пустых записей в таблице
df[(df['Ответ GPT'].isna())].head(10) #проверка пустых записей в таблице

In [ ]:
df[['Вопрос', 'Ответ GPT', 'Ссылки', 'Время ответа', 'Чанки']].iloc[23:28]     # вывод строк с 7 по 12

In [ ]:
df.to_excel('/content/test.xlsx')      # загрузка df в таблицу 'test.xlsx'

In [ ]:
# если результат плохой, можно править код или промпты прямо в колабе в папке слева, при этом сервер перезапускать не нужно.
# потом нужно сохранить измененные файлы на Github.


In [ ]:
!pkill uvicorn    #Для завершения всех фоновых процессов
ngrok.kill()